In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [12]:
#Importing all the dependencies
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Reshape, Flatten, Lambda, Conv2DTranspose,UpSampling2D
from keras.preprocessing import backend as K
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import mnist
from keras import optimizers

In [3]:
# download training and test data from mnist and reshape it

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = X_train.astype('float32') / 255.
output_X_train = X_train.reshape(-1,28,28,1)

X_test = X_test.astype('float32') / 255.
output_X_test = X_test.reshape(-1,28,28,1)

print(X_train.shape, X_test.shape)

11493376/11490434 [==============================] - 6s 0us/step
(60000, 28, 28) (10000, 28, 28)


In [4]:
# adding some noise to data

input_x_train = output_X_train + 0.5 * np.random.normal(loc=0.0, scale=1.0, size=output_X_train.shape) 
input_x_test = output_X_test + 0.5 * np.random.normal(loc=0.0, scale=1.0, size=output_X_test.shape)

In [13]:
input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (7, 7, 32)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

In [15]:
autoencoder = Model(input_img, decoded)
m = 128
n_epoch = 50
adam = optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
autoencoder.compile(optimizer=adam, loss='binary_crossentropy',metrics=['accuracy'])
hist = autoencoder.fit(input_x_train,output_X_train, epochs=n_epoch, batch_size=m, shuffle=True,validation_split=0.20)

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 3s 64us/step - loss: 0.2954 - accuracy: 0.8033 - val_loss: 0.1615 - val_accuracy: 0.8037
Epoch 2/50
48000/48000 [==============================] - 3s 54us/step - loss: 0.1442 - accuracy: 0.8062 - val_loss: 0.1340 - val_accuracy: 0.8087
Epoch 3/50
48000/48000 [==============================] - 3s 56us/step - loss: 0.1278 - accuracy: 0.8087 - val_loss: 0.1244 - val_accuracy: 0.8099
Epoch 4/50
48000/48000 [==============================] - 3s 54us/step - loss: 0.1206 - accuracy: 0.8098 - val_loss: 0.1186 - val_accuracy: 0.8108
Epoch 5/50
48000/48000 [==============================] - 3s 55us/step - loss: 0.1160 - accuracy: 0.8106 - val_loss: 0.1149 - val_accuracy: 0.8114
Epoch 6/50
48000/48000 [==============================] - 3s 54us/step - loss: 0.1130 - accuracy: 0.8110 - val_loss: 0.1126 - val_accuracy: 0.8116
Epoch 7/50
48000/48000 [==============================] - 3s 57us/st

In [20]:
decoded_imgs = autoencoder.predict(input_x_test,verbose=0)



Test loss: [[[2.85877577e-05]
  [7.98769520e-07]
  [1.85723920e-07]
  [6.76978615e-08]
  [3.39013795e-08]
  [1.52269006e-08]
  [1.10284848e-08]
  [8.42408721e-09]
  [1.07531584e-08]
  [9.18886123e-09]
  [6.56275034e-09]
  [5.80113690e-09]
  [4.08629885e-09]
  [4.34241532e-09]
  [5.70836711e-09]
  [8.94673935e-09]
  [1.16237429e-08]
  [1.30292275e-08]
  [1.59723612e-08]
  [2.21995933e-08]
  [4.11945784e-08]
  [1.06447025e-07]
  [1.89153610e-07]
  [2.46219770e-07]
  [3.06665470e-07]
  [1.07945652e-06]
  [3.29216959e-06]
  [4.15523755e-05]]

 [[1.75772948e-05]
  [2.71060117e-06]
  [5.21307811e-07]
  [1.19740406e-07]
  [5.24305221e-08]
  [3.48319915e-08]
  [6.13277891e-08]
  [9.44380503e-08]
  [1.28863320e-07]
  [1.16043971e-07]
  [9.26729555e-08]
  [8.45311021e-08]
  [4.92068821e-08]
  [5.18335277e-08]
  [7.27237435e-08]
  [1.20168124e-07]
  [1.80889174e-07]
  [3.04752916e-07]
  [4.21388847e-07]
  [6.71802525e-07]
  [1.42936801e-06]
  [1.95150483e-06]
  [1.50803692e-06]
  [9.15105772e-07]

In [ ]:
# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

In [ ]:
# summarize history for accuracy
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

In [ ]:
n = 15
plt.figure(figsize=(20, 4))
for i in range(1,n):
    # display original noisy image
    ax = plt.subplot(2, n, i)
    plt.imshow(input_x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display denosined image
    ax = plt.subplot(2, n, i + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()